In [ ]:
import sys
sys.path.append("../src")

from generate_heatmap import *

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

%matplotlib inline

result_dir = '../results/figures/'

PLOTTING FOR UNFILTERED DATA CASE

In [ ]:
%store -r data_unfiltered

In [ ]:
plot_heatmap(data_unfiltered, result_dir)

PLOTTING FOR FILTERED CASE

In [ ]:
%store -r data_filtered

In [ ]:
plot_heatmap(data_filtered, result_dir)